In [1]:
import pandas as pd
import numpy as np
import re

In [19]:
mtg_df=pd.read_csv('..\mtg_data.csv', index_col=0)
mtg_df=mtg_df.dropna(subset='oracle_text')
mtg_df.head(5)

,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,rarity,flavor_text,text,text_prompt,card_description
0,Fury Sliver,{5}{R},6.0,Creature — Sliver,All Sliver creatures have double strike.,3,3,['R'],['R'],[],uncommon,"""A rift opened, and our arrows were abruptly s...",Fury Sliver: [SEP] {5}{R} [SEP] Creature — Sli...,Fury Sliver: [SEP] {5}{R},Creature — Sliver [SEP] All Sliver creatures h...
1,Kor Outfitter,{W}{W},2.0,Creature — Kor Soldier,"When ~ enters the battlefield, you may attach ...",2,2,['W'],['W'],[],common,"""We take only what we need to survive. Believe...",Kor Outfitter: [SEP] {W}{W} [SEP] Creature — K...,Kor Outfitter: [SEP] {W}{W},Creature — Kor Soldier [SEP] When ~ enters the...
2,Spirit,NaN,0.0,Token Creature — Spirit,Flying,1,1,['W'],['W'],[Flying],common,NaN,Spirit: [SEP] [SEP] Token Creature — Spirit [...,Spirit: [SEP],Token Creature — Spirit [SEP] Flying
3,Siren Lookout,{2}{U},3.0,Creature — Siren Pirate,"Flying\nWhen ~ enters the battlefield, it expl...",1,2,['U'],['U'],"[Flying, Explore]",common,NaN,Siren Lookout: [SEP] {2}{U} [SEP] Creature — S...,Siren Lookout: [SEP] {2}{U},Creature — Siren Pirate [SEP] Flying\nWhen ~ e...
4,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cas...,NaN,NaN,['G'],['G'],[Enchant],rare,NaN,Web: [SEP] {G} [SEP] Enchantment — Aura [SEP] ...,Web: [SEP] {G},Enchantment — Aura [SEP] Enchant creature (Tar...


In [20]:
#pre-processing to get rid of unregonizable characters
rare_char={
    '¡®°²½˝̶π’„•…™−∞☐œŠ':'',
    'Äàáâãä':'a',
    'Éèéêë':'e',
    'Ææ':'ae',
    'Óóö':'o',
    'úûü':'u',
    'íī':'i',
    'Ññ':'n'
}
for rarechar, target in rare_char.items():
    for char in [*rarechar]:
        mtg_df['oracle_text']=mtg_df['oracle_text'].str.replace(char, target)

In [34]:
text_list=[text.lower() for text in list(mtg_df['oracle_text'])]
text_len=np.array([len(desc) for desc in text_list])
print(
    'total number of characters:\n'+
    str(np.sum(text_len))+'\n'+
    'average numbr of characters:\n'+
    str(np.mean(text_len))+'\n'+
    'total number of cards:\n'+
    str(len(text_len))
)

total number of characters:
10926627
average numbr of characters:
136.95775936626515
total number of cards:
79781


In [35]:
max(len(s) for s in text_list)

763

In [47]:
#training a tokenizer just for the cards
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

#pre-split into items seperated by whitespaces
tokenizer.pre_tokenizer = Whitespace()

#pre-processing the strings to create special tokens

tokenizer.train_from_iterator(text_list, trainer=trainer)

vocab_size=tokenizer.get_vocab_size()
print(f'vocab size:{vocab_size}')
tokenizer.save('mtggenerator.dict')
output=tokenizer.encode('[CLS]')
print(output.tokens)
print(output.ids)
print(tokenizer.decode(output.ids))

vocab size:7300
['[CLS]']
[1]



In [49]:
tokenizer.get_vocab()

{'ninja': 1704,
 'mutate': 748,
 'semifinal': 5748,
 'logic': 6785,
 'ory': 6170,
 'narfi': 6898,
 '}."': 1010,
 'disenchant': 3978,
 'ery': 349,
 'pumped': 7126,
 'article': 6314,
 'sin': 6002,
 'scavengers': 4528,
 'system': 6600,
 'mer': 923,
 'cstatic': 5862,
 'loyal': 3670,
 'commander': 621,
 'yanggu': 4203,
 'she': 2589,
 'hor': 844,
 'ney': 4824,
 'ile': 194,
 'information': 3767,
 'ade': 3264,
 'formidable': 2258,
 'exert': 1452,
 'killbot': 5627,
 'ink': 483,
 'lost': 920,
 'neurotraumal': 5797,
 'writhing': 4660,
 'worlds': 2673,
 'fencer': 5897,
 'reve': 310,
 'respon': 4896,
 'bill': 4722,
 'advisors': 2747,
 'fur': 3281,
 'quill': 3968,
 'muster': 3055,
 'apes': 6606,
 'ei': 820,
 'strefan': 4672,
 'd': 38,
 'represent': 1078,
 'lifelink': 490,
 'n': 48,
 'simultaneously': 2232,
 'achieved': 7210,
 'retriever': 3766,
 'others': 3093,
 'everyone': 4043,
 'exper': 1291,
 'acere': 4358,
 'allure': 5098,
 'trium': 5220,
 'ochre': 5690,
 'okaun': 3393,
 'ludevic': 6668,
 'ange

In [50]:
#training a tokenizer just for the cards
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "{U}", "{W}", "{R}", "{B}", "{G}", "{T}", "{X}"])

#pre-split into items seperated by whitespaces
tokenizer.pre_tokenizer = Whitespace()

#pre-processing the strings to create special tokens

tokenizer.train_from_iterator(text_list, trainer=trainer)

vocab_size=tokenizer.get_vocab_size()
print(f'vocab size:{vocab_size}')
tokenizer.save('mtggenerator_tokenizer_v6.json')

vocab size:7306


In [53]:
#create the mapping from characters to integers
encode = lambda text: tokenizer.encode(text).ids #encode: take a string, output a list of integers
decode = lambda list: tokenizer.decode(list) #decode: take a list of integers, output a string

#test
print(encode('[CLS] Black Lotus'))
print(decode(encode('when battlefield')))

[1, 0, 52, 185, 0, 374, 571]
when battlefield


In [8]:
import torch
from torch.nn.utils.rnn import pad_sequence

#convert data to 2d tensor
encoded_text_list=[torch.Tensor(encode(text)) for text in text_list]
max_len=max([len(item) for item in encoded_text_list])
padded_text_list=[torch.cat((item, torch.full((max_len - len(item),), 3))) for item in encoded_text_list] # the [PAD] token has id=3
padded_text_list_with_CLS = [torch.cat((torch.tensor([1]), item)) for item in padded_text_list]

data = pad_sequence(padded_text_list_with_CLS, batch_first=True).long() # N_cards * Char_length
data.shape

torch.Size([85036, 191])

In [9]:
#train test split

n_train=int(0.9*data.shape[0])
train_data=data[:n_train]
val_data=data[n_train:]
print(train_data.shape)
print(val_data.shape)

torch.Size([76532, 191])
torch.Size([8504, 191])


In [10]:
block_size=8
train_data[0][:block_size+1]

tensor([1821,  762,   29,  127,   91,  762,  844,  762,  246])

In [11]:
x = train_data[0][:block_size]
y = train_data[0][1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f'when input is {context} the target is {target}')

when input is tensor([1821]) the target is 762
when input is tensor([1821,  762]) the target is 29
when input is tensor([1821,  762,   29]) the target is 127
when input is tensor([1821,  762,   29,  127]) the target is 91
when input is tensor([1821,  762,   29,  127,   91]) the target is 762
when input is tensor([1821,  762,   29,  127,   91,  762]) the target is 844
when input is tensor([1821,  762,   29,  127,   91,  762,  844]) the target is 762
when input is tensor([1821,  762,   29,  127,   91,  762,  844,  762]) the target is 246


In [87]:
torch.manual_seed(69)
batch_size=4
block_size=8

def get_batch(split):
    #generates a small batch of data input x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.stack([torch.randint(data.shape[0], (batch_size, )), torch.randint(256 - block_size, (batch_size, ))]).T
    x = torch.stack(tuple(data[i[0]][i[1]:i[1] + block_size] for i in ix))
    y = torch.stack(tuple(data[i[0]][i[1] + 1:i[1] + block_size + 1] for i in ix))
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----------')

inputs:
torch.Size([4, 8])
tensor([[69, 86, 69, 82,  1, 65,  1, 67],
        [83, 13,  1, 69, 65, 67, 72,  1],
        [76, 76,  1, 84, 72, 73, 83,  1],
        [73, 78, 83,  1, 70, 65, 76, 76]])
targets:
torch.Size([4, 8])
tensor([[86, 69, 82,  1, 65,  1, 67, 82],
        [13,  1, 69, 65, 67, 72,  1, 79],
        [76,  1, 84, 72, 73, 83,  1, 84],
        [78, 83,  1, 70, 65, 76, 76, 73]])
-----------


In [96]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(69)

class BigranLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, vocab_size) #each token directly look up the logit of the next token from a lookup table
    
    def forward(self, idx, targets=None):
        #idx and targets are both (B,T) tensors of integers, where B=batch number, T=position in batch
        logits=self.token_embedding_table(idx) #look up value corresponding to own position in the token embedding table to form C (channel value)

        if targets is None:
            loss=None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #logits are therefore values associated with each character
            loss=F.cross_entropy(logits, targets) #evaluate loss

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #use logits only, focus only on last time step
            logits = logits[:, -1, :] #keep only last time step ---> (B, C)
            #apply softmax on logit to get distribution
            probs = F.softmax(logits, dim=-1) #get a (B, C) matrix of probabilities, sum(prob) of each B = 1
            #sample from the distribution
            idx_next=torch.multinomial(probs, num_samples=1) #get a (B, 1) array of predictions
            #append prediction to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #now a (B, T+1) matrix of returned results
        return idx

m=BigranLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 94])
tensor(4.9992, grad_fn=<NllLossBackward0>)


In [97]:
#testing baseline model
print(decode(m.generate(idx= torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


f)&f
nSa)O(]q3pXWx)lQbVt1M|i{13aqAI@rQlf\QY<%3JJe*FdY\@J9]+.SNn"-vUue_?bxOT+
e?yErcQJ*p$jT!E(wwb-I{Y


In [ ]:
#testing baseline model
print(decode(m.generate(idx= torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


f)&f
nSa)O(]q3pXWx)lQbVt1M|i{13aqAI@rQlf\QY<%3JJe*FdY\@J9]+.SNn"-vUue_?bxOT+
e?yErcQJ*p$jT!E(wwb-I{Y


In [ ]:
#create new optimizer
optimizer=torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size=32
for steps in range(10000):

    #sample a batch of data
    xb, yb = get_batch('train')
    #evaluate the loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    print(loss.item())

3.156108856201172
2.9403269290924072
2.9307777881622314
3.0349888801574707
2.83091402053833
3.0683441162109375
2.9718310832977295
2.978175163269043
3.154308795928955
3.015265464782715
3.0164496898651123
3.029155969619751
3.03064227104187
2.9563684463500977
3.0332372188568115
2.9494097232818604
2.957733392715454
2.972060203552246
3.0173397064208984
3.127647876739502
3.1646697521209717
2.9095873832702637
3.0046000480651855
2.9531455039978027
2.81375789642334
2.879044771194458
2.973694324493408
3.002920150756836
3.0172855854034424
2.998901605606079
2.9897243976593018
3.0411458015441895
3.043302059173584
2.8867175579071045
2.9628360271453857
3.050635576248169
3.070373773574829
3.138213634490967
2.963299036026001
2.9558074474334717
2.9889349937438965
2.9258551597595215
3.1600279808044434
2.9973440170288086
2.892343282699585
3.112966299057007
3.040086269378662
2.9953770637512207
2.9258923530578613
2.8760368824005127
2.9151451587677
2.7869863510131836
3.0890300273895264
2.9415576457977295
2.9

In [ ]:
print(decode(m.generate(idx= torch.zeros((1,1), dtype=torch.long), max_new_tokens=250)[0].tolist()))


Tourent  atr.
+1.
Le     citra ge:=Micr        And an Add l: or    hand,    hedist   t  Crdd:
Sok            Lou     Sls   tstey    lachafl   is  turnge  Pldsinge wesive,     o any  cky      celachas  "
Ityird."tseticrou      rea     y: uely,     d +
